In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rioxr

from pystac_client import Client  # To access STAC catalogs

import planetary_computer  # To sign items from the MPC STAC catalog 

from IPython.display import Image  # To nicely display images

In [2]:
# Access MPC catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
bii = catalog.get_child('io-biodiversity')

In [22]:
range_2017 = "2017-01-01/2017-12-31"
range_2020 = "2020-01-01/2020-12-31"


bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-112.826843, 32.974108],
        [-111.184387, 32.974108],
        [-111.184387, 33.863574],
        [-112.826843, 33.863574],
        [-112.826843, 32.974108]  # close polygon
    ]]
}

In [23]:
search_2017 = catalog.search(
    collections = ['io-biodiversity'],
    intersects = bbox,
    datetime = range_2017)

search_2017

In [24]:
# Retrieve search items
items = search_2017.item_collection()
len(items)

1

In [ ]:
polygon = {
    "type": "Polygon",
    "coordinates": [[
        [-112.826843, 32.974108],
        [-111.184387, 32.974108],
        [-111.184387, 33.863574],
        [-112.826843, 33.863574],
        [-112.826843, 32.974108]  # close polygon
    ]]
}

search_2017 = catalog.search(
    collections=["io-biodiversity"],
    intersects=polygon,
    datetime="2017-01-01/2017-12-31"
)
items = search_2017.item_collection()


In [13]:
items = search_2017.item_collection()
len(items)

1

In [14]:
item = items[0]


In [25]:
# Print item ID and properties
print('ID:' , item.id)
item.properties

ID: bii_2017_34.74464974521749_-115.38597824385106_cog


{'datetime': None,
 'proj:shape': [7992, 7992],
 'end_datetime': '2017-12-31T23:59:59Z',
 'proj:transform': [0.0008983152841195215,
  0.0,
  -115.38597824385106,
  0.0,
  -0.0008983152841195215,
  34.74464974521749,
  0.0,
  0.0,
  1.0],
 'start_datetime': '2017-01-01T00:00:00Z',
 'proj:code': 'EPSG:4326'}

In [26]:
 item.assets

{'data': <Asset href=https://pcdata01euw.blob.core.windows.net/impact/bii-v1/bii_2017/bii_2017_34.74464974521749_-115.38597824385106_cog.tif?st=2025-12-01T14%3A23%3A53Z&se=2025-12-02T15%3A08%3A53Z&sp=rl&sv=2025-07-05&sr=c&skoid=9c8ff44a-6a2c-4dfb-b298-1c9212f64d9a&sktid=72f988bf-86f1-41af-91ab-2d7cd011db47&skt=2025-12-02T04%3A20%3A53Z&ske=2025-12-09T04%3A20%3A53Z&sks=b&skv=2025-07-05&sig=MvsoAYKCIkK0zkOUu6Oe/8KMHbiGrBVFXBQJ4osm5zA%3D>,
 'tilejson': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/tilejson.json?collection=io-biodiversity&item=bii_2017_34.74464974521749_-115.38597824385106_cog&assets=data&tile_format=png&colormap_name=io-bii&rescale=0%2C1&expression=0.97%2A%28data_b1%2A%2A3.84%29&format=png>,
 'rendered_preview': <Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=io-biodiversity&item=bii_2017_34.74464974521749_-115.38597824385106_cog&assets=data&tile_format=png&colormap_name=io-bii&rescale=0%2C1&expression=0.97

In [27]:
for key in item.assets.keys():
    print(key, '--', item.assets[key].title)

data -- Biodiversity Intactness
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [28]:
# Plot rendered preview
Image(url=item.assets['rendered_preview'].href, width=500)

In [29]:
pheonix = rioxr.open_rasterio(item.assets['data'].href)

In [ ]:
fp = os.path.join('data','tl_2020_04_cousub','tl_2020_04_cousub.shp')
counties = gpd.read_file(fp)

In [ ]:
counties.head()

In [30]:
pheonix.head()

<xarray.DataArray (band: 1, y: 5, x: 5)> Size: 100B
[25 values with dtype=float32]
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 40B -115.4 -115.4 -115.4 -115.4 -115.4
  * y            (y) float64 40B 34.74 34.74 34.74 34.74 34.74
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [31]:
print(type(pheonix.values))

<class 'numpy.ndarray'>


In [32]:
print("Size of dimensions: ", dict(pheonix.sizes))

Size of dimensions:  {'band': 1, 'y': 7992, 'x': 7992}


In [33]:
print('CRS: ', pheonix.rio.crs)

CRS:  EPSG:4326
